<a href="https://colab.research.google.com/github/annikabrundyn/da_vinci/blob/main/make_exp_table.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import itertools
import openpyxl
import random
import time

In [21]:
### save to spreadsheet

from google.colab import auth
auth.authenticate_user()

import gspread
import pandas as pd
from google.colab import auth
from oauth2client.client import GoogleCredentials
from gspread_dataframe import set_with_dataframe, get_as_dataframe

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [22]:
sh = gc.open('qual_exp_table')

In [23]:
### read from sheets

img_comps = sh.worksheet('img_comps')
items = sh.worksheet('items')

In [24]:
### get the models, images and videos we'll compare

model_names = sh.values_get("'items'!A5:A50")['values']
img_names = sh.values_get("'items'!B5:B50")['values']
video_names = sh.values_get("'items'!C5:C50")['values']

# flatten lists
model_names = sum(model_names, [])
img_names = sum(img_names, [])
video_names = sum(video_names, [])

In [25]:
print(model_names)
print(img_names)

['s1', 's3', 's8', 'u3', 'u13', 'u18', 'u19', 'u22', 'u26']
['0', '1', '12', '13']


In [58]:
### generate image comparisons between models

model_combs = list(itertools.combinations(model_names, 2))

# shuffle order of a/b
random.seed(3478)
model_combs = [random.sample(list(comb), len(comb)) for comb in model_combs]

In [59]:
### model + frame combinations

model_frame_combs = list(itertools.product(model_combs, img_names))
model_frame_combs = [(a, b, frame) for ((a, b), frame) in model_frame_combs]

In [60]:
model_frame_combs[0:10]

[('s3', 's1', '0'),
 ('s3', 's1', '1'),
 ('s3', 's1', '12'),
 ('s3', 's1', '13'),
 ('s1', 's8', '0'),
 ('s1', 's8', '1'),
 ('s1', 's8', '12'),
 ('s1', 's8', '13'),
 ('s1', 'u3', '0'),
 ('s1', 'u3', '1')]

In [61]:
### add in sanity checks

sanity_checks = [('u22', 'u3', '0'),
                 ('u3', 'u22', '0'),
                 ('u22', 'u3', '12'),
                 ('u3', 'u22', '12')]

In [63]:
model_frame_combs += sanity_checks

In [64]:
model_frame_combs[-10:]

[('u26', 'u19', '12'),
 ('u26', 'u19', '13'),
 ('u26', 'u22', '0'),
 ('u26', 'u22', '1'),
 ('u26', 'u22', '12'),
 ('u26', 'u22', '13'),
 ('u22', 'u3', '0'),
 ('u3', 'u22', '0'),
 ('u22', 'u3', '12'),
 ('u3', 'u22', '12')]

In [65]:
### add in comparisons with target:
target_frame_num = 0
target_comps = []

for model in model_names:
  # randomly determine order
  if random.random() < 0.5:
    a = model
    b = "target"
  else:
    a = "target"
    b = model
  
  target_comps.append((a, b, target_frame_num))

In [66]:
target_comps

[('target', 's1', 0),
 ('s3', 'target', 0),
 ('s8', 'target', 0),
 ('target', 'u3', 0),
 ('target', 'u13', 0),
 ('target', 'u18', 0),
 ('target', 'u19', 0),
 ('u22', 'target', 0),
 ('target', 'u26', 0)]

In [67]:
model_frame_combs += target_comps

In [68]:
model_frame_combs[-10:]

[('u3', 'u22', '12'),
 ('target', 's1', 0),
 ('s3', 'target', 0),
 ('s8', 'target', 0),
 ('target', 'u3', 0),
 ('target', 'u13', 0),
 ('target', 'u18', 0),
 ('target', 'u19', 0),
 ('u22', 'target', 0),
 ('target', 'u26', 0)]

In [69]:
random.seed(14)
random.shuffle(model_frame_combs)

In [70]:
model_frame_combs

[('u26', 'u13', '12'),
 ('u26', 'u22', '1'),
 ('u18', 'u26', '0'),
 ('u22', 'u3', '12'),
 ('s3', 'u13', '0'),
 ('s1', 's8', '1'),
 ('s1', 'u3', '12'),
 ('u3', 'u13', '0'),
 ('u26', 'u22', '0'),
 ('u18', 's3', '0'),
 ('u3', 's3', '1'),
 ('u26', 'u3', '12'),
 ('s8', 'u22', '0'),
 ('u26', 'u13', '1'),
 ('u18', 'u19', '13'),
 ('u18', 'u19', '12'),
 ('u22', 'target', 0),
 ('u26', 's1', '1'),
 ('u22', 'u3', '13'),
 ('s1', 'u3', '0'),
 ('u22', 'u3', '1'),
 ('s3', 'u13', '13'),
 ('u18', 's3', '13'),
 ('u26', 'u19', '0'),
 ('s8', 'target', 0),
 ('u22', 'u18', '1'),
 ('s3', 's1', '0'),
 ('u18', 'u19', '0'),
 ('u26', 'u19', '13'),
 ('u18', 'u13', '13'),
 ('u3', 'u13', '1'),
 ('u3', 'u22', '0'),
 ('s1', 's8', '12'),
 ('u22', 'u19', '13'),
 ('u22', 'u19', '1'),
 ('u13', 'u22', '1'),
 ('u26', 'u19', '1'),
 ('u19', 'u13', '0'),
 ('s3', 's8', '1'),
 ('u18', 's1', '13'),
 ('s3', 'target', 0),
 ('u18', 's1', '0'),
 ('target', 'u19', 0),
 ('u18', 'u13', '12'),
 ('u26', 's8', '13'),
 ('u19', 's1', '12'),


In [71]:
sh.values_clear("'img_comps'!A5:J10000")

{'clearedRange': 'img_comps!A5:J1000',
 'spreadsheetId': '1pIUn3doGKx15l43OFiuIiMGRmY9RzCaIfJRhEnXz8nU'}

In [72]:
row = 5
quest_num = 1
worksheet = img_comps

for comb in model_frame_combs:
  a, b, frame = comb

  worksheet.update_cell(row, 1, quest_num)
  worksheet.update_cell(row, 2, a)
  worksheet.update_cell(row, 3, b)
  worksheet.update_cell(row, 4, frame)

  row += 1
  quest_num += 1
  time.sleep(2)